## Data Interoperability Ex

Goals:
- compare pandas to polars
- work with parquet files

In [1]:
import pandas as pd
import polars as pl
import parquet as pq

In [2]:
source = "https://raw.githubusercontent.com/kenf1/ITTF-Autoscrape/main/data/SEN_MD-data.csv"

### Import csv

- read csv from website

In [3]:
%%time
pandas_df = pd.read_csv(source)
pandas_df.head()

CPU times: user 28.2 ms, sys: 2.31 ms, total: 30.5 ms
Wall time: 231 ms


,Rank Only,Rank Change,Name,Assoc,Points
0,1,NaN,FAN Zhendong / WANG Chuqin,CHN,6040
1,2,NaN,JANG Woojin / LIM Jonghoon,KOR,4390
2,3,NaN,Yukiya UDA / Shunsuke TOGAMI,JPN,3555
3,4,NaN,Dang QIU / Benedikt DUDA,GER,1895
4,5,NaN,CHO Seungmin / AN Jaehyun,KOR,1515


In [4]:
%%time
polars_df = pl.read_csv(source)
polars_df.head()

CPU times: user 40.8 ms, sys: 6.73 ms, total: 47.6 ms
Wall time: 280 ms


Rank Only,Rank Change,Name,Assoc,Points
i64,str,str,str,i64
1,null,"""FAN Zhendong /…","""CHN""",6040
2,null,"""JANG Woojin / …","""KOR""",4390
3,null,"""Yukiya UDA / S…","""JPN""",3555
4,null,"""Dang QIU / Ben…","""GER""",1895
5,null,"""CHO Seungmin /…","""KOR""",1515


### Multiple filters

filter dataframe by:
- ranks 1:30 (include 30)
- country

In [5]:
%%time
new_pandas_df = pandas_df.loc[(pandas_df["Rank Only"]<=30) & (pandas_df["Assoc"] == "CHN")]
new_pandas_df.head()

CPU times: user 789 µs, sys: 2 ms, total: 2.79 ms
Wall time: 2.2 ms


,Rank Only,Rank Change,Name,Assoc,Points
0,1,NaN,FAN Zhendong / WANG Chuqin,CHN,6040
11,12,NaN,LIN Gaoyuan / LIN Shidong,CHN,950
14,15,NaN,CHEN Yuanyu / LIN Shidong,CHN,769
22,23,NaN,XIANG Peng / LIN Shidong,CHN,594
23,24,NaN,YUAN Licen / XIANG Peng,CHN,570


In [6]:
%%time
new_polars_df = polars_df.filter(
    (pl.col("Rank Only")<=30) & (pl.col("Assoc")=="CHN")
)
new_polars_df.head()

CPU times: user 456 µs, sys: 4 ms, total: 4.45 ms
Wall time: 2.53 ms


Rank Only,Rank Change,Name,Assoc,Points
i64,str,str,str,i64
1,null,"""FAN Zhendong /…","""CHN""",6040
12,null,"""LIN Gaoyuan / …","""CHN""",950
15,null,"""CHEN Yuanyu / …","""CHN""",769
23,null,"""XIANG Peng / L…","""CHN""",594
24,null,"""YUAN Licen / X…","""CHN""",570


### Write to parquet

- using pyarrow because it is default for pandas [source](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_parquet.html)

In [7]:
class outputs:
    home_dir = "../data/"
    pd = home_dir+"pd"+".parquet"
    pl = home_dir+"pl"+".parquet"

In [8]:
%%time
new_pandas_df.to_parquet(outputs.pd)

CPU times: user 14.1 ms, sys: 2.52 ms, total: 16.7 ms
Wall time: 34.1 ms


In [9]:
%%time
new_polars_df.write_parquet(outputs.pl)

CPU times: user 3.96 ms, sys: 12.8 ms, total: 16.8 ms
Wall time: 172 ms


### Read from parquet

In [10]:
%%time
new_pd = pd.read_parquet(outputs.pd)

CPU times: user 5.47 ms, sys: 4.38 ms, total: 9.85 ms
Wall time: 7.63 ms


In [11]:
%%time
new_pl = pl.read_parquet(outputs.pl)

CPU times: user 1.05 ms, sys: 193 µs, total: 1.24 ms
Wall time: 3.67 ms


## Conclusion

In almost all cases, [polars](https://www.pola.rs/) is faster than [pandas](https://pandas.pydata.org/). The syntax is very similar to pandas and thus does not require much to convert current and future workflows over to polars.